<a href="https://colab.research.google.com/github/RaquelGrosman/BigD/blob/master/Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Libraries imported
import numpy.linalg
import numpy as np
import scipy.integrate
import scipy.misc
import scipy.special
import math
import cmath as cm
import sympy as sp
import matplotlib.pyplot as plt

#sensitivity curve - Power Spectra
#setting variables 
f = 215 #set this ne shanges not from paper
f0 = 150 #Hz
fs = 40 #Hz lower bound freq
S0 = (9*10**-46) #Hz -1

#defining constants
c = 3*10**8
G = 6.674*10**-11
epsilon = 0.1

#the gravitational waves
#setting variables
SNR = 10
m1 = 10**30 * 9.109 * 10**(-31) #arbitrary (solar masses)
m2 = 10**30 * 9.109 * 10**(-31)#arbitrary
M = m1 + m2
q= m2/m1

#alpha values
tc = 1 #arbitrary
phi = 1 #arbitrary
eta = (m1*m2)/(M**2)
v = (math.pi*M*f)**1/3

#setting alpha values
alpha = []
alpha.append(1)
alpha.append(0)

parameters = [0, 0, 0, 0]


In [0]:
## Thanks to felix and Szymon; the following functions will be used for the waveforms, 
## using func_index to pick the type of input

def psi_f(f, parameters, func_index):
    try:
        if func_index == 1:
            return psi_f1(f, parameters)
        elif func_index == 2:
            return psi_f2(f, parameters)
        else:
            return cause_an_error
    except:
        print("Does NOT exist")

#used for exercies 1, 2 and 3
def psi_f1(f, parameters):
    phi, tc, M, eta = parameters
    v = (np.pi * M * f)**(-1 / 3)
    sum_k = 0
    for k in range(len(alpha)):
        sum_k += alpha[k] * v**k
    return 2 * np.pi * f * t - phi - np.pi / 4 + 3 / (128 * eta * v**5) * sum_k

#for exercise 4: incluedes resonance frequency/phase shift - uses epsilon instead of phi
def psi_f2(f, parameters):
    epsilon, tc, M, eta = parameters
    f = get_resonance_freq()
    v = (np.pi * M * f)**(-1 / 3)
    sum_k = 0
    for k in range(len(alpha)):
        sum_k += alpha[k] * v**k
    return 2 * np.pi * f * t - get_phase_shift(epsilon) - np.pi / 4 + 3 / (128 * eta * v**5) * sum_k

# using epsilon get k1
def get_k1(epsilon):
    return (8/(5*(7+3*math.log(epsilon))))

#calculating phase shift of g-wave
def get_phase_shift(epsilon):
    return ((25*np.pi*0.1**2)/(6144/2))*(np.abs(get_k1(epsilon))/(q*(1+q)))

#calculating resonance F of black hole
def get_resonance_freq():
    return ((c**3)/(m1*G))*(1/np.abs(math.log(epsilon)))

#find A (integral from low to high f^-7/6)/S(f)df
def get_A(freq_list, pds, SNR):
    integral = 0
    k = 0
    for f, s_h in zip(freq_list, pds):
        if k < (len(freq_list)-1):
            integral += ((f)**(-7 / 6) / s_h)*(freq_list[k+1]-f)
            k += 1
    return np.sqrt(SNR / 4*abs(integral))

#calculate h
def get_h_f(A, f, parameters, function_index):
    psi = psi_f(f, parameters, function_index)
    h_f = A * f**(-7 / 6) * np.exp(1j * psi)
    return h_f


In [0]:
#calling/getting/loading Einstein data
#credit to Felix and Szymon, thanks for your guidance

def load_EPDS():
    try:
        file = open('EinsteinPDS')
    except:
        print("No data found")
    string_read = file.read()
    arr = list(map(float, string_read.split()))
    x_arr = []
    y_arr1 = []
    y_arr2 = []
    y_arr3 = []
    for i in range(len(arr)):
        if i % 4 == 0:
            x_arr.append(arr[i])
        if i % 4 == 1:
            y_arr1.append(arr[i])
        if i % 4 == 2:
            y_arr2.append(arr[i])
        if i % 4 == 3:
            y_arr3.append(arr[i])
    plt.plot(x_arr, y_arr1)
    plt.plot(x_arr, y_arr2)
    plt.plot(x_arr, y_arr3)
    plt.yscale('log')
    plt.xscale('log')
    return x_arr, y_arr3


In [0]:
#calling/getting/loading LIGO data
def load_LIGO():
    try:
        file = open('LigoPDS')
    except:
        print("No data found")
    string_read = file.read()
    arr = list(map(float, string_read.split()))
    x_arr = []
    y_arr1 = []
    y_arr2 = []
    y_arr3 = []
    y_arr4 = []
    y_arr5 = []
    y_arr6 = []
    for i in range(len(arr)):
        if i % 7 == 0:
            x_arr.append(arr[i])
        if i % 7 == 1:
            y_arr1.append(arr[i])
        if i % 7 == 2:
            y_arr2.append(arr[i])
        if i % 7 == 3:
            y_arr3.append(arr[i])
        if i % 7 == 4:
            y_arr4.append(arr[i])
        if i % 7 == 5:
            y_arr5.append(arr[i])
        if i % 7 == 6:
            y_arr6.append(arr[i])
    plt.plot(x_arr, y_arr1)
    plt.plot(x_arr, y_arr2)
    plt.plot(x_arr, y_arr3)
    plt.plot(x_arr, y_arr4)
    plt.plot(x_arr, y_arr5)
    plt.plot(x_arr, y_arr6)
    plt.yscale('log')
    plt.xscale('log')
    return x_arr, y_arr6

In [0]:
#calculates sensitivity curve for initial LIGO
def s_h_LIGO(f):
    S0 = 9 * 10**(-46)
    fs = 40
    f0 = 150
    x = f / f0
    if(f >= fs):
        r = S0 * (((4.49 * x)**(-56)) + 0.16 *
                   (x**(-4.52)) + 0.52 + (0.32 * (x**2)))
    else:
        r = 10000000  # if infinity leads to numerical errors
    return(r)

In [0]:
#calculates sensitivity curve for advanced LIGO
def s_h_adv_LIGO(f):
    f0 = 215
    fs = 20
    S0 = 6 * 10**(-49)
    x = f / f0
    if (f >= fs):
        r = S0 * (x**(-4.14) - 5 * x**(-2) +
                   ((111 * (1 - x**2 + (x**4) / 2) / (1 + (x**2) / 2))))
    else:
        r = 10000000  # if infinity leads to numerical errors
    return(r)

In [0]:
#h'
def h_der(A, f, i, func_index):
  dx = parameters[i]/10 
  newP = parameters.copy()
  newP[i] = newP[i] + parameters[i]/10
  dy = get_h_f(A, f, newP, func_index) - get_h_f(A, f, parameters, func_index)
  return dx/dy


In [0]:
#matrix creation using above created functions
def create_fisher_matrix(A, freq_list, pds, func_index):
    fisher_matrix = []
    for i in range(len(parameters)):
        fisher_matrix.append([])
        for j in range(len(parameters)):
            integral = 0
            k = 0
            for f, sh in zip(freq_list, pds):
                if k < (len(freq_list)-1):
                    integral += np.real(h_der(A, f, i, func_index)
                                        * np.conj(h_der(A, f, j, func_index)) / sh)*(freq_list[k+1]-f)
                    k += 1
                # print(integral)
            fisher_matrix[-1].append(4*integral)
    return(fisher_matrix)

In [0]:

#function to plot PDS for LIGO inital and Advanced
def graph_s_h(telescope):
    graph_array = []
    try:
        if telescope == 'LIGO':
            for i in range(0, 1000):
                graph_array.append(np.sqrt(s_h_LIGO(i)))
        elif telescope == 'adv_LIGO':
            for i in range(0, 1000):
                graph_array.append(np.sqrt(s_h_adv_LIGO(i)))
        plt.figure()
        plt.plot(graph_array)
        plt.yscale('log')
        plt.xscale('log')
        axes = plt.gca()
        axes.set_ylim([10**(-24), 10**(-21)])
        axes.set_xlim([10**1, 10**3])
    except:
        print("Computation not avaiable")

In [0]:
# pull and read PDS of advanced ligo and Einstein telescope

freq_list, pds = load_LIGO()
freq_list_einstein, pds_einstein = load_EPDS()

# EPDS is capable of detecting smaller (lower freqency) signals, frequency of 10^1 and less
# x-axis does not have the same range!!
# LIGO stops measuring accurately around 10^2 and lower



In [0]:
# parameters are set
parameters = [phi, tc, M, eta]
# integration to auire A
A = get_A(freq_list, pds, SNR)
A_einstein = get_A(freq_list_einstein, pds, SNR)

print('A of LIGO: ', A)
print('A of Einstein Telescope: ', A_einstein)


matrix = create_fisher_matrix(A, freq_list, pds, 1)

matrix_einstein = create_fisher_matrix(A, freq_list_einstein, pds_einstein, 1)

# invert fisher_matrix
inverse = np.linalg.inv(matrix)
inverse_einstein = np.linalg.inv(matrix_einstein)

# extract diagonal and square root
diag = np.abs(np.diagonal(inverse))
diag_einstein = np.abs(np.diagonal(inverse_einstein))
print('Diagonal of LIGO fishermatrix: ', diag)
print('Diagonal of Einstein Telescope fishermatrix: ', diag_einstein)
std = np.sqrt(diag)
std_einstein = np.sqrt(diag_einstein)

# Print standard deviation and standard deviation/value
print('Standard deviation of LIGO: ', std)
print('Standard deviation of Einstein Telescope: ', std_einstein)

# Parameters are in the following order: phi, tc, M, eta
print('Print values of parameters: ', parameters)
print('Percent of LIGO: ', np.divide(std, parameters))
print('Percent of Einstein Telescope: ', np.divide(std_einstein, parameters))

In [0]:
#excersise 4 : looking at the quantum black hole - using epsilon

parameters = [epsilon, tc, M, eta]
# now, a is set via integration
A = get_A(freq_list, pds, SNR)
A_einstein = get_A(freq_list_einstein, pds, SNR)

print('A of LIGO: ', A)
print('A of Einstein Telescope: ', A_einstein)

matrix = create_fisher_matrix(A, freq_list, pds, 1)

matrix_einstein = create_fisher_matrix(A, freq_list_einstein, pds_einstein, 1)

# invert fisher_matrix
inverse = np.linalg.inv(matrix)
inverse_einstein = np.linalg.inv(matrix_einstein)

# extract diagonal and square root
diag = np.abs(np.diagonal(inverse))
diag_einstein = np.abs(np.diagonal(inverse_einstein))
print('diagonal of LIGO fishermatrix: ', diag)
print('diagonal of Einstein Telescope fishermatrix: ', diag_einstein)
std = np.sqrt(diag)
std_einstein = np.sqrt(diag_einstein)

# Print standard deviation and standard deviation/value
print('Standard deviation of LIGO: ',std)
print('Standard deviation of Einstein Telescope: ', std_einstein)

# parameters are in the following order: epsilon, tc, M, eta
print('Print values of parameters: ', parameters)
print('Percent of LIGO: ', np.divide(std, parameters))
print('Percent of Einstein Telescope: ', np.divide(std_einstein, parameters))

In [0]:
## again super props to Felix and Szymon for this <3
### This section will run for quite a while since the fisher matrix has
### to be computed many times. It returns the epsilon standard deviation
### depending on epsilon itself and the mass ratio (q)

parameters = [epsilon, tc, M, eta]

A_einstein = get_A(freq_list_einstein, pds, SNR)

m1 = 1.4*(1.988*10**30 * 9.109 * 10**(-31)) #Setting mass 1 to be 1.4 solar masses
color_plot = []


for i in range(0,11):
    m_two = ((i+1)*10)*(1.988*10**30 * 9.109 * 10**(-31))
    for j in range(0,11):
        epsilon = (j*10+1650)/100000
        matrix_einstein = create_fisher_matrix(A_einstein, freq_list_einstein, pds_einstein, 2)
        inverse_einstein = np.linalg.inv(matrix_einstein)
        diag_einstein = np.abs(np.diagonal(inverse_einstein))
        std_einstein = np.sqrt(diag_einstein)
        color_plot.append(std_einstein[0])
    print("loop ran")

In [0]:
#plot
#X:epsilon/y:mass ratio; Stdev of epsilon represented by colors: the bright the higher

x = np.linspace(-1, 1, 11)
y = np.linspace(-1, 1, 11)

X, Y = np.meshgrid(x, y)
Z = np.array(color_plot).reshape(11,11)
plt.pcolor(X, Y, Z)
plt.imshow(Z, origin='lower',interpolation='bilinear')

In [0]:
#plots inital and adv ligo data - from given fomulas (3.7 and 3.8 from arxiv paper)

graph_s_h('LIGO')
graph_s_h('adv_LIGO')